# Keras Tuner Bars

In [1]:
import h5py
import numpy as np
import tensorflow
from tensorflow import keras
import kerastuner

print("tensorflow", tensorflow.__version__)
print("keras", keras.__version__)
print("keras-tuner", kerastuner.__version__)

tensorflow 2.1.0
keras 2.2.4-tf
keras-tuner 1.0.1


In [2]:
class HDF5DataGeneratorBars(keras.utils.Sequence):
    def __init__(self, file, num_classes, batch_size=200, mode="train", train_test_split=0.8):
        self.file = h5py.File(file, "r")
        self.num_classes = num_classes
        self.batch_size = batch_size
        self.mode = mode

        batches = int(np.floor(self.file["multiplicity"].len() / self.batch_size))
        if self.mode == "train":
            self.num_batches = int(batches * train_test_split)
            self.offset = 0
        elif self.mode == "test":
            self.num_batches = int(batches * (1 - train_test_split))
            self.offset = int(batches * train_test_split)
        else:
            raise

    def __len__(self):
        return self.num_batches

    def __getitem__(self, index):
        x, y = self.getbatch(index)
        np.place(y, y >= self.num_classes, self.num_classes - 1)  # FIXME
        y = keras.utils.to_categorical(y, num_classes=self.num_classes)
        return x, y

    def getbatch(self, index):
        "Generate one batch of data"
        start = self.batch_size * (index + self.offset)
        end = self.batch_size * (index + self.offset + 1)
        
        # all the bars PLUS the trifeatures
        x1 = np.array(self.file["flatfeatures"][start:end])
        x2 = np.array(self.file["consolidated"][start:end])
        x = np.append(x1, x2, axis=1)
        
        # multiplicity column 2 -> label = 'nPH'
        y = np.array(self.file["multiplicity"][start:end, [2]].ravel())
        
        return x, y

In [3]:
class Model(kerastuner.HyperModel):
    def __init__(self, num_classes):
        self.num_classes = num_classes

    def build(self, hp):
        model = keras.models.Sequential()
        for i in range(hp.Int("num_layers", 1, 2)):
            model.add(
                keras.layers.Dense(
                    units=hp.Int(
                        f"units_{i}", min_value=1000, max_value=10000, step=3000
                    ),
                    activation="relu",
                )
            )
        model.add(
            keras.layers.Dense(
                units=hp.Int(f"units_last", min_value=50, max_value=550, step=100),
                activation="relu",
            )
        )
        model.add(keras.layers.Dense(units=self.num_classes, activation="softmax"))

        optimizer = hp.Choice('optimizer', ['adam', 'adagrad', 'adadelta', 'SGD'])
        loss = keras.losses.CategoricalCrossentropy()

        model.compile(loss=loss, optimizer=optimizer, metrics=["accuracy"])

        return model

In [4]:
num_dp = 30
max_neutrons = 5
num_classes = max_neutrons + 1

file = f"data/600AMeV_{num_dp}dp.bars-shuffled.h5"
generator = HDF5DataGeneratorBars(file, num_classes=num_classes, mode='train')
test_generator = HDF5DataGeneratorBars(file, num_classes=num_classes, mode='test')
model = Model(num_classes)

#tuner = kerastuner.tuners.BayesianOptimization(
tuner = kerastuner.tuners.RandomSearch(
    model,
    objective="val_accuracy",
    max_trials=20,
    executions_per_trial=1,
    directory="tmp-kerastuner",
    project_name="tunebars10",
)
tuner.search_space_summary()

INFO:tensorflow:Reloading Oracle from existing project tmp-kerastuner\tunebars10\oracle.json
INFO:tensorflow:Reloading Tuner from tmp-kerastuner\tunebars10\tuner0.json


In [5]:
tuner.search(
    generator,
    validation_data = test_generator,
    verbose=2,
)

  ...
    to  
  ['...']
Train for 20000 steps, validate for 4999 steps
20000/20000 - 3852s - loss: 19177617.7845 - accuracy: 0.5575 - val_loss: 23872475.7071 - val_accuracy: 0.6228


  ...
    to  
  ['...']
Train for 20000 steps, validate for 4999 steps
20000/20000 - 1146s - loss: nan - accuracy: 0.0050 - val_loss: nan - val_accuracy: 0.0049


c:\users\jan\.virtualenvs\futurereco2-s-fscxbi\lib\site-packages\kerastuner\engine\metrics_tracking.py:92: RuntimeWarning: All-NaN axis encountered
  return np.nanmin(values)


  ...
    to  
  ['...']
Train for 20000 steps, validate for 4999 steps
20000/20000 - 1794s - loss: 417412.0853 - accuracy: 0.3315 - val_loss: 259066.2258 - val_accuracy: 0.3973


  ...
    to  
  ['...']
Train for 20000 steps, validate for 4999 steps
20000/20000 - 1197s - loss: nan - accuracy: 0.0050 - val_loss: nan - val_accuracy: 0.0049


...

  ...
    to  
  ['...']
Train for 20000 steps, validate for 4999 steps
20000/20000 - 1224s - loss: 5675058.7401 - accuracy: 0.4157 - val_loss: 3743044.3561 - val_accuracy: 0.4600


  ...
    to  
  ['...']
Train for 20000 steps, validate for 4999 steps
20000/20000 - 1134s - loss: 837965.5291 - accuracy: 0.4844 - val_loss: 808865.7126 - val_accuracy: 0.5410


  ...
    to  
  ['...']
Train for 20000 steps, validate for 4999 steps
20000/20000 - 984s - loss: nan - accuracy: 0.0050 - val_loss: nan - val_accuracy: 0.0049


c:\users\jan\.virtualenvs\futurereco2-s-fscxbi\lib\site-packages\kerastuner\engine\metrics_tracking.py:92: RuntimeWarning: All-NaN axis encountered
  return np.nanmin(values)


INFO:tensorflow:Oracle triggered exit


In [6]:
tuner.results_summary()